In [0]:
import pyspark.sql.functions as sf
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
unix_time_window = time() - 86400

In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
df_map = df_map.filter("ID IN ('1679','1603','1619','2357','444')")

In [0]:
df_map.display()

In [0]:
# Read data from 'runescape.02_silver.latest_prices_enriched'
# and filter to the last 12 hours
df = spark.read.table("runescape.02_silver.latest_prices_enriched").filter(f"time >= {unix_time_window}")
df = df.filter("time >= 1770192286 AND time <= 1770194757")
df = df.withColumn("NYCTime", sf.from_unixtime('time'))
df = df.filter("ID IN ('1679','1603','1619','2357','444')")
df = df.sort("time")
df.count()

In [0]:
df_ruby_cutting_enriched=df.filter("ID IN ('1603','1619')") #Ruby, Uncut Ruby
df_ruby_cutting_enriched.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# Read data from 'runescape.01_bronze.latest_prices_raw'
dfraw_ruby = spark.read.table("runescape.01_bronze.latest_prices_raw")

# Read data from 'runescape.01_bronze.latest_prices_raw'
dfraw_ruby = dfraw_ruby.filter("ID IN ('1603','1619')") #Ruby, Uncut Ruby
dfraw_ruby = dfraw_ruby.filter(f"time >= {unix_time_window}") # exclude older data
dfraw_ruby = dfraw_ruby.filter("time >= 1770192286 AND time <= 1770194757")
dfraw_ruby = dfraw_ruby.sort("time")
#df2 = df2.dropDuplicates()
#df2 = df2.sort("NYCTime", ascending=False)
dfraw_ruby.display()

Databricks visualization. Run in Databricks to view.

In [0]:
dfraw_ruby2 = dfraw_ruby.dropDuplicates()
dfraw_ruby2.display()

In [0]:
# Read data from 'runescape.02_silver.latest_prices_cleansed'
# and filter to the last 12 hours
df_cleansed = spark.read.table("runescape.02_silver.latest_prices_cleansed")
df_cleansed = df_cleansed.filter("time >= 1770192286 AND time <= 1770194757")
df_cleansed = df_cleansed.withColumn("NYCTime", sf.from_unixtime('time'))
df_cleansed = df_cleansed.filter("ID IN ('1603','1619')")
df_cleansed = df_cleansed.sort("time")
df_cleansed.count()

In [0]:
df_cleansed.display()

In [0]:
# Read data from 'runescape.01_bronze.1h_prices' to get Volumes
# TODO replace this read with the new latest_1_prices table that will have the latest 1 hour price for each item
df_avg = spark.read.table("runescape.01_bronze.1h_prices")

In [0]:
# Get average of 1 hour price data over the last 24 hours
# TODO replace this with actual 24 hour calculation based on underlying raw data
df_24_hours_avg = df_avg.\
    filter(f"time >= {unix_time_window}").\
    filter("ID IN ('1679','1603','1619','2357','444')").\
    groupBy("id").\
    agg(sf.round(sf.avg("avg1HourHigh"), 2).alias("avg_24hr_High")
    , sf.round(sf.avg("avg1HourHighVolume"), 2).alias("avg_24hr_High_Volume")
    , sf.round(sf.avg("avg1HourLow"), 2).alias("avg_24hr_Low")
    , sf.round(sf.avg("avg1HourLowVolume"), 2).alias("avg_24hr_Low_Volume"))




In [0]:
%skip

# Filter to only use the most recent Volume data
max_time = df_avg.select(sf.max(df_avg.time))

#df_avg = df_avg.filter("time == max_time.first('time'")
df_avg = df_avg.join(max_time, df_avg.time == sf.col("max(time)"))
df_avg = df_avg.drop("max(time)").drop("time")
#df_avg.display()




In [0]:
df2 = df
# filter out low limit items
# df2 = df2.filter(sf.col("limit") >= 70)
# filter to only sell data
# df2 = df2.filter(sf.col("highorlow") == "high")

In [0]:

df3 = df2.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("price")) - 100,1))
# Join with 24 hour data to get volume
df3 = df3.join(df_24_hours_avg, "id",)
#df3 = df3.filter("avg_24hr_High_Volume >= 100")
#df3 = df3.filter("high_Alch_Margin >= 400")
#df3 = df3.sort("high_Alch_Margin", ascending=False)
df3.display()

TODO figure out how to make a chart of the difference from/ profit margin between uncut and cut rubys

In [0]:
df_ruby_amulet = df3.filter("id == 1679")
df_ruby_amulet.display()


Databricks visualization. Run in Databricks to view.

In [0]:
df_ruby = df3.filter("id == 1603")
df_ruby.display()


Databricks visualization. Run in Databricks to view.

In [0]:
df_unct_ruby = df3.filter("id == 1619")
df_unct_ruby.display()


Databricks visualization. Run in Databricks to view.

In [0]:
df_ruby_cutting = df3.filter("id IN ('1619','1603')")
df_ruby_cutting.display()


Databricks visualization. Run in Databricks to view.

In [0]:
df_gold_bar = df3.filter("id == 2357")
df_gold_bar.display()


Databricks visualization. Run in Databricks to view.

In [0]:
%skip
df10 = df9.groupBy("id","highorlow").agg(sf.round(sf.avg("price"), 2).alias("avg_price"), sf.round(sf.std("price"), 2).alias("std_price"))
#join with volume data
df10 = df10.join(df_avg, "id").withColumn("std/avg", sf.round(sf.col("std_price")/sf.col("avg_price"), 3))
-df10 = df10.join(df_map,"id")
df10 = df10.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("avg_price")) - 100,1))
df10 = df10.withColumn("price-1hour_high_price", sf.round(sf.col("avg_price") - sf.col("avg1HourHigh") , 2))
df10 = df10.withColumn("limit_X_std_price", sf.col("std_price")*sf.col("limit"))



df10 = df10.filter("highorlow = 'low'")
df10 = df10.filter("std_price >= 2")
df10 = df10.filter("avg1HourHighVolume > 1000")
df10 = df10.sort("std/avg", ascending=False)
df10.display()

In [0]:
%skip
df8.display()